In [ ]:
# get page ranges

#  round datetime to day, get ± 7 days inclusive

#  merge any date ranges that overlaps

#  reorganise data into dict where key is date range and value is a list of pages

In [1]:
import pandas as pd
from urllib.parse import unquote
import wikitoolkit as wt
import pickle
import sqlite3
import time
import os

In [2]:
my_agent = 'wikireddit <p.gildersleve@exeter.ac.uk>'
conn = sqlite3.connect('data/wikireddit.db')

ranges_df = pd.read_hdf('data/ranges_df.h5')

with open('data/langpagemaps.pkl', 'rb') as f:
    pagemapsdict = pickle.load(f)

In [3]:
ranges_df_consol = ranges_df.groupby(['lang', 'start_date', 'end_date'])['title'].apply(list)
date_article_dict = {}
# convert ranges_df_consol to a dict of dicts
for lang, start_date, end_date in ranges_df_consol.index:
    if lang not in date_article_dict:
        date_article_dict[lang] = {}
    date_article_dict[lang][(start_date, end_date)] = ranges_df_consol[(lang, start_date, end_date)]

starts_df_consol = ranges_df.groupby(['lang', 'start_date'])['title'].apply(list)
startdate_article_dict = {}
# convert starts_df_consol to a dict of dicts
for lang, start_date in starts_df_consol.index:
    if lang not in startdate_article_dict:
        startdate_article_dict[lang] = {}
    startdate_article_dict[lang][start_date] = starts_df_consol[(lang, start_date)]

Get revisions in range [from, to] (inclusive), as well as revisions at start of range

In [ ]:
if os.path.exists('data/done_langdateranges_rev.pkl'):
    with open('data/done_langdateranges_rev.pkl', 'rb') as f:
        done_langdateranges = pickle.load(f)
else:
    done_langdateranges = {}

maxgroupsize = 10000
for lang, da_dict in date_article_dict.items():
    print()
    try:
        l_revisions_df = [pd.read_hdf('data/revisions.h5', key=f'/{lang}')]
    except (KeyError, FileNotFoundError):
        l_revisions_df = []
    if lang not in done_langdateranges:
        done_langdateranges[lang] = set()
    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    for n, (daterange, articles) in enumerate(da_dict.items()):
        if n % 100 == 0:
            print(lang, n/len(da_dict), end='\r')
        if daterange in done_langdateranges[lang]:
            continue

        grouprevs = {}
        groupsize = maxgroupsize
        while len(articles) > 0:
            try:
                revisions = await wt.get_revisions(wtsession, articles[:groupsize], pagemaps=pagemapsdict[lang],
                                                start=daterange[0].isoformat(), stop=daterange[1].isoformat())
                grouprevs.update(revisions)
                articles = articles[groupsize:]
                groupsize = min(int(round(groupsize * (2**0.6), 0)), maxgroupsize)
            except ValueError as e:
                print(e, 'Reducing group size to', groupsize // 2)
                time.sleep(0.1)
                groupsize = groupsize // 2

        revisions = pd.concat({k: pd.DataFrame(v) for k, v in grouprevs.items()}).reset_index(
            level=1, drop=True).reset_index().rename(columns={'index': 'title'})
        done_langdateranges[lang].add(daterange)
        if len(revisions) == 0:
            continue
        revisions['lang'] = lang
        revisions = revisions[['lang', 'title', 'revid', 'parentid', 'timestamp']]
        l_revisions_df.append(revisions)

        if (n%5000 == 0)&(n > 0): # save every 5000 for very large langs for safety
            if len(l_revisions_df) > 0:
                l_revisions_df_i = pd.concat(l_revisions_df, ignore_index=True)
                l_revisions_df_i.to_hdf('data/revisions.h5', key=f'/{lang}', mode='a')
                del l_revisions_df_i
                with open('data/done_langdateranges_rev.pkl', 'wb') as f:
                    pickle.dump(done_langdateranges, f)
        
    if len(l_revisions_df) > 0:
        l_revisions_df = pd.concat(l_revisions_df, ignore_index=True)
        l_revisions_df.to_hdf('data/revisions.h5', key=f'/{lang}', mode='a')
        with open('data/done_langdateranges_rev.pkl', 'wb') as f:
            pickle.dump(done_langdateranges, f)
    await wtsession.close()


aa 0.0
ab 0.0
ace 0.0
ady 0.0
af 0.0
als 0.0
am 0.0
an 0.0
ang 0.0
ar 0.94899169632265726
arc 0.0
ary 0.0
arz 0.0
as 0.0
ast 0.0
atj 0.0
avk 0.0
ay 0.0
az 0.90497737556561093
azb 0.0
ba 0.0
ban 0.0
bar 0.7633587786259542
bat-smg 0.0
bcl 0.0
be 0.0
be-tarask 0.0
bg 0.96830985915492966
bh 0.0
bi 0.0
bjn 0.0
bn 0.0
bo 0.0
bpy 0.0
br 0.0
bs 0.73529411764705894


/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'bat-smg'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'be-tarask'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)



bxr 0.0
ca 0.89418777943368115
cbk-zam 0.0
cdo 0.0
ce 0.0
ceb 0.0
chr 0.0
chy 0.0
ckb 0.0
co 0.0
cr 0.0
crh 0.0
cs 0.993227990970654634
csb 0.0
cu 0.0
cv 0.0
cy 0.9259259259259259


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7c5b83731dc0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7c5b76562120>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7c5acc5729f0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7c5b8ff85280>


da 0.99585062240663913


In [22]:
parentids = set(pd.concat(l_revisions_df, ignore_index=True)['parentid'].astype(int).unique())
revids = set(pd.concat(l_revisions_df, ignore_index=True)['revid'].astype(int).unique())

In [25]:
len(parentids), len(revids), len(parentids - revids)

(32720, 32723, 1635)

In [ ]:
# get starting revids from parentid column - this won't work for everything
for lang in date_article_dict:
    l_revisions_df = pd.read_hdf('data/revisions.h5', key=f'/{lang}')
    parentids = set(l_revisions_df['parentid'].unique())
    revids = set(l_revisions_df['revid'].unique())

    start_revids = parentids - revids
    start_revids = list(start_revids - set([0]))
    while len(start_revids) > 0:
        try:
            revisions = await 
        
    

In [15]:
maxgroupsize = 10000
for lang, da_dict in startdate_article_dict.items():
    print(lang)
    l_revisions_df = []
    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()
    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    for n, (date, articles) in enumerate(da_dict.items()):
        if n % 1000 == 0:
            print(n/len(da_dict), end='\r')
        grouprevs = {}
        groupsize = maxgroupsize
        while len(articles) > 0:
            try:
                revisions = await wt.get_revision(wtsession, titles=articles[:groupsize], pagemaps=pagemapsdict[lang],
                                                 date=date.isoformat())
                grouprevs.update(revisions)
                articles = articles[groupsize:]
                groupsize = min(int(round(groupsize * (2**0.6), 0)), maxgroupsize)
            except ValueError as e:
                print(e, 'Reducing group size to', groupsize // 2)
                time.sleep(0.1)
                groupsize = groupsize // 2

        date_revisions = pd.DataFrame({k: v for k, v in grouprevs.items() if v}).T.reset_index().rename(columns={'index': 'title'})
        if len(date_revisions) == 0:
            continue
        date_revisions['lang'] = lang
        date_revisions = date_revisions[['lang', 'title', 'revid', 'parentid', 'timestamp']]
        l_revisions_df.append(date_revisions)

    if len(l_revisions_df) > 0:
        l_revisions_df = pd.concat(l_revisions_df, ignore_index=True)
        l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')
    await wtsession.close()

aa


/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


ab


/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


ace


...

/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


be-tarask


/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'be-tarask'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


bg


CancelledError: 

quality

topics

In [3]:
lang_title_dict = ranges_df[['lang', 'title']].drop_duplicates().groupby('lang')['title'].apply(list).to_dict()

maxgroupsize = 1000
l_topics_df = []
for lang, articles in lang_title_dict.items():

    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()

    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)

    grouptopics = {}
    groupsize = maxgroupsize
    while len(articles) > 0:
        print(f'{lang}: {len(articles)} articles remaining. {len(grouptopics)} articles processed.', end='\r')
        try:
            # a_topics = await wt.get_articles_topics(wtsession, articles[:groupsize],
            #                                      lang=lang, tf_args={'threshold': 0},
            #                                      pagemaps=pagemapsdict[lang])
            a_topics = wt.get_articles_topics_sync(wtsession, articles[:groupsize],
                                        lang=lang, tf_args={'threshold': 0},
                                        pagemaps=pagemapsdict[lang])
            grouptopics.update(a_topics)
            articles = articles[groupsize:]
            groupsize = min(int(round(groupsize * (2**0.6), 0)), maxgroupsize)
        except ValueError as e:
            print(e, 'Reducing group size to', groupsize // 2)
            time.sleep(0.1)
            groupsize = groupsize // 2

    grouptopics = pd.concat({k: pd.Series(v) for k, v in grouptopics.items()}).reset_index().rename(
                    columns={'level_0': 'article', 'level_1': 'topic', 0: 'score'})
    grouptopics = grouptopics.pivot(index='article', columns='topic', values='score').reset_index().rename_axis(None, axis=1)
    grouptopics['lang'] = lang
    grouptopics = grouptopics.set_index(['lang', 'article'])
    l_topics_df.append(grouptopics)
    
    await wtsession.close()

l_topics_df = pd.concat(l_topics_df, ignore_index=True)

: 

: 

pageviews

In [5]:
if os.path.exists('data/done_langdateranges_pv.pkl'):
    with open('data/done_langdateranges_pv.pkl', 'rb') as f:
        done_langdateranges = pickle.load(f)
else:
    done_langdateranges = {}

maxgroupsize = 10000
for lang, da_dict in date_article_dict.items():
    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()
    if lang not in done_langdateranges:
        done_langdateranges[lang] = set()
    try:
        l_pvdf = [pd.read_hdf('data/date_pageviews.h5', key=f'/{lang}')]
        print(lang, 'successful read')
    except (KeyError, FileNotFoundError):
        l_pvdf = []

    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    
    for n, (daterange, articles) in enumerate(da_dict.items()):
        if n % 100 == 0:
            print(lang, n/len(da_dict))
        if daterange in done_langdateranges[lang]:
            continue
        grouppvs = []
        groupsize = maxgroupsize
        while len(articles) > 0:
            try:
                try:
                    pageviews = wt.api_article_views(wtsession, f'{lang}.wikipedia', articles[:groupsize], pagemaps=pagemapsdict[lang],
                            start=daterange[0].strftime('%Y%m%d'), end=(daterange[1] - pd.Timedelta(days=1)).strftime('%Y%m%d'))
                except Exception as e:
                    if e.args[0][:44] == 'The pageview API returned nothing useful at:':
                        pageviews = {d.to_pydatetime(): {x: 0 for x in articles}
                                for d in pd.date_range(start=daterange[0], end=daterange[1] - pd.Timedelta(days=1))}
                    else:
                        raise e
                    
                pvdf = pd.DataFrame(pageviews).T.reset_index().rename(columns={'index': 'date'})
                pvdf = pvdf.melt(id_vars='date', var_name='title', value_name='pageviews')
                pvdf['lang'] = lang
                pvdf = pvdf[['lang', 'title', 'date', 'pageviews']]
                grouppvs.append(pvdf)
                articles = articles[groupsize:]
                groupsize = min(int(round(groupsize * (2**0.6), 0)), maxgroupsize)
            except ValueError as e:
                print(e, 'Reducing group size to', groupsize // 2)
                time.sleep(0.1)
                groupsize = groupsize // 2
        gpvdf = pd.concat(grouppvs, ignore_index=True)
        l_pvdf.append(gpvdf)
        done_langdateranges[lang].add(daterange)

        if (len(l_pvdf) > 0) & (n % 1000 == 0) & (n > 0):
            print('writing intermediate')
            l_pvdfw = pd.concat(l_pvdf, ignore_index=True)
            l_pvdfw.to_hdf('data/date_pageviews.h5', key=f'/{lang}', mode='a')
            with open('data/done_langdateranges_pv.pkl', 'wb') as f:
                pickle.dump(done_langdateranges, f)

    if len(l_pvdf) > 0:
        l_pvdf = pd.concat(l_pvdf, ignore_index=True)
        l_pvdf.to_hdf('data/date_pageviews.h5', key=f'/{lang}', mode='a')
        with open('data/done_langdateranges_pv.pkl', 'wb') as f:
            pickle.dump(done_langdateranges, f)        
    await wtsession.close()

aa successful read
aa 0.0
ab successful read
ab 0.0
ace successful read
ace 0.0
ady successful read
ady 0.0


af successful read
af 0.0
ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/af.wikipedia/all-access/all-agents/Melianthus/daily/2021120200/2021122300']


Traceback (most recent call last):
  File "/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/mwviews/api/pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/af.wikipedia/all-access/all-agents/Melianthus/daily/2021120200/2021122300']


als successful read
als 0.0


CancelledError: 

Wikidata IDs